<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/26_prompt_engineering/26_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install -qq ibm-watsonx-ai
# ! pip install -qq langchain
# ! pip install -qq langchain-ibm
# ! pip install -qq git+https://github.com/ibm-granite-community/utils

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# In-Context Engineering and Prompt Templates

## Setup LLM

In [8]:
from langchain_ibm.llms import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from google.colab import userdata

def llm_model(prompt_txt, params=None):
    model_id = 'mistralai/mixtral-8x7b-instruct-v01'

    default_params = {
        "max_new_tokens": 256,
        "min_new_tokens": 0,
        "temperature": 0.5,
        "top_p": 0.2,
        "top_k": 1
    }

    if params:
        default_params.update(params)

    parameters = {
        GenParams.MAX_NEW_TOKENS: default_params["max_new_tokens"],  # this controls the maximum number of tokens in the generated output
        GenParams.MIN_NEW_TOKENS: default_params["min_new_tokens"], # this controls the minimum number of tokens in the generated output
        GenParams.TEMPERATURE: default_params["temperature"], # this randomness or creativity of the model's responses
        GenParams.TOP_P: default_params["top_p"],
        GenParams.TOP_K: default_params["top_k"]
    }

    api_key = userdata.get('IBM_CLOUD_API_KEY')

    credentials = {
      "url": userdata.get("WATSONX_URL"),
      "apikey": userdata.get('IBM_CLOUD_API_KEY')
  }

    project_id = userdata.get("WATSONX_PROJECT_ID")

    mixtral_llm = WatsonxLLM(
        model_id=model_id,
        url=userdata.get("WATSONX_URL"),
        apikey=userdata.get('IBM_CLOUD_API_KEY'),
        project_id=userdata.get("WATSONX_PROJECT_ID"),
        params=parameters
    )

    response  = mixtral_llm.invoke(prompt_txt)
    return response

In [9]:
GenParams().get_example_values()

{'decoding_method': 'sample',
 'length_penalty': {'decay_factor': 2.5, 'start_index': 5},
 'temperature': 0.5,
 'top_p': 0.2,
 'top_k': 1,
 'random_seed': 33,
 'repetition_penalty': 2,
 'min_new_tokens': 50,
 'max_new_tokens': 200,
 'stop_sequences': ['fail'],
 'time_limit': 600000,
 'truncate_input_tokens': 200,
 'prompt_variables': {'object': 'brain'},
 'return_options': {'input_text': True,
  'generated_tokens': True,
  'input_tokens': True,
  'token_logprobs': True,
  'token_ranks': False,
  'top_n_tokens': False}}

## Prompt engineering

In [10]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 10,
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1
}

prompt = "The wind is"

response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'mistralai/mixtral-8x7b-instruct-v01' is in deprecated state from 2025-04-30 until 2025-07-30. IDs of alternative models: mistralai/mistral-small-3-1-24b-instruct-2503. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


prompt: The wind is

response :  howling outside, and the rain is coming down in sheets. It’s the perfect weather for a cozy night in with a good book. But what to read? If you’re looking for something to take your mind off the storm, here are five great books to curl up with on a rainy night.

## 1. The Secret History by Donna Tartt

This gripping novel tells the story of a group of classics students at an elite New England college who become embroiled in a murder conspiracy. With its richly drawn characters and atmospheric setting, The Secret History is

